# Projeto 3 - Classificação binária breast cancer com tuning dos parâmetros

## 1. Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np
import torch.nn as nn
from skorch import NeuralNetBinaryClassifier
import torch
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV


In [2]:
torch.set_default_device("mps")
device = torch.device("mps")

# torch.set_default_device("cpu")
# device = torch.device("cpu")

In [3]:
print(torch.backends.mps.is_available())  # Deve retornar True
print(torch.backends.mps.is_built())  # Deve retornar True

True
True


## 2. Importando Dados

In [4]:
np.random.seed(123)
torch.manual_seed(123)

In [5]:
previsores = pd.read_csv("data/entradas_breast.csv")
classe = pd.read_csv("data/saidas_breast.csv")

In [6]:
previsores = np.array(previsores, dtype='float32')
classe = np.array(classe, dtype='float32').squeeze(1)

## 3. Classe para a rede neural

In [7]:
class classificador_torch(nn.Module):
    def __init__(self, activation, neurons, initializer):
        super().__init__()

        # 30 -> 16 -> 16 -> 1
        self.dense0 = nn.Linear(30, neurons)
        initializer(self.dense0.weight)
        self.activation0 = activation

        self.dense1 = nn.Linear(neurons, neurons)
        initializer(self.dense1.weight)
        self.activation1 = activation

        self.dense2 = nn.Linear(neurons, 1)
        initializer(self.dense2.weight)

        self.output = nn.Sigmoid()

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation0(X)

        X = self.dense1(X)
        X = self.activation1(X)

        X = self.dense2(X)

        X = self.output(X)
        return X

## 4. Skorch

In [8]:
classificador_sklearn = NeuralNetBinaryClassifier(module=classificador_torch,
                                                  lr=0.001,
                                                  optimizer__weight_decay=0.0001,
                                                  train_split=False)

## 5. Tuning dos parâmetros

In [9]:
params = {
    'batch_size': [10], #30
    'max_epochs': [50], #100
    'optimizer': [torch.optim.Adam], #torch.optim.SGD],
    'criterion': [torch.nn.BCELoss], #torch.nn.HingeEmbeddingLoss],
    'module__activation': [F.relu, F.tanh],
    'module__neurons': [8, 16],
    'module__initializer': [torch.nn.init.uniform_, torch.nn.init.normal_]
}

In [10]:
grid_search = GridSearchCV(estimator=classificador_sklearn,
                          param_grid=params,
                          scoring='accuracy',
                          cv=2)
grid_search = grid_search.fit(previsores, classe)

  epoch    train_loss     dur
-------  ------------  ------
      1       37.3239  0.1218
      2       37.3239  0.0900
      3       37.3239  0.0833
      4       37.3239  0.0795
      5       37.3239  0.1211
      6       37.3239  0.0858
      7       37.3239  0.1079
      8       37.3239  0.0936
      9       37.3239  0.1060
     10       37.3239  0.0860
     11       37.3239  0.0838
     12       37.3239  0.0898
     13       37.3239  0.0915
     14       37.3239  0.0938
     15       37.3239  0.0847
     16       37.3239  0.0932
     17       37.3239  0.0930
     18       37.3239  0.0956
     19       37.3239  0.0850
     20       37.3239  0.0876
     21       37.3239  0.0851
     22       37.3239  0.0835
     23       37.3239  0.0819
     24       37.3239  0.0794
     25       37.3239  0.0840
     26       37.3239  0.0840
     27       37.3239  0.0874
     28       37.3239  0.0806
     29       37.3239  0.0795
     30       37.3239  0.0806
     31       37.3239  0.0813
     32   

In [11]:
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_

In [12]:
melhores_parametros

{'batch_size': 10,
 'criterion': torch.nn.modules.loss.BCELoss,
 'max_epochs': 50,
 'module__activation': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
 'module__initializer': <function torch.nn.init.uniform_(tensor: torch.Tensor, a: float = 0.0, b: float = 1.0, generator: Optional[torch._C.Generator] = None) -> torch.Tensor>,
 'module__neurons': 8,
 'optimizer': torch.optim.adam.Adam}

In [13]:
melhor_precisao

np.float64(0.7436125525080306)